In [50]:
%logstop
%logstart -rtq ~/.logs/pw.py append
%matplotlib inline
import matplotlib
import seaborn as sns
sns.set()
matplotlib.rcParams['figure.dpi'] = 144

In [3]:
from static_grader import grader

# PW Miniproject
## Introduction

The objective of this miniproject is to exercise your ability to use basic Python data structures, define functions, and control program flow. We will be using these concepts to perform some fundamental data wrangling tasks such as joining data sets together, splitting data into groups, and aggregating data into summary statistics.
**Please do not use `pandas` or `numpy` to answer these questions.**

We will be working with medical data from the British NHS on prescription drugs. Since this is real data, it contains many ambiguities that we will need to confront in our analysis. This is commonplace in data science, and is one of the lessons you will learn in this miniproject.

## Downloading the data

We first need to download the data we'll be using from Amazon S3:

In [4]:
%%bash
mkdir pw-data
wget http://dataincubator-wqu.s3.amazonaws.com/pwdata/201701scripts_sample.json.gz -nc -P ./pw-data
wget http://dataincubator-wqu.s3.amazonaws.com/pwdata/practices.json.gz -nc -P ./pw-data

mkdir: cannot create directory ‘pw-data’: File exists
File ‘./pw-data/201701scripts_sample.json.gz’ already there; not retrieving.

File ‘./pw-data/practices.json.gz’ already there; not retrieving.



## Loading the data

The first step of the project is to read in the data. We will discuss reading and writing various kinds of files later in the course, but the code below should get you started.

In [5]:
import gzip
import simplejson as json

In [6]:
with gzip.open('./pw-data/201701scripts_sample.json.gz', 'rb') as f:
    scripts = json.load(f)

with gzip.open('./pw-data/practices.json.gz', 'rb') as f:
    practices = json.load(f)

This data set comes from Britain's National Health Service. The `scripts` variable is a list of prescriptions issued by NHS doctors. Each prescription is represented by a dictionary with various data fields: `'practice'`, `'bnf_code'`, `'bnf_name'`, `'quantity'`, `'items'`, `'nic'`, and `'act_cost'`. 

In [7]:
scripts[:2]

[{'bnf_code': '0101010G0AAABAB',
  'items': 2,
  'practice': 'N81013',
  'bnf_name': 'Co-Magaldrox_Susp 195mg/220mg/5ml S/F',
  'nic': 5.98,
  'act_cost': 5.56,
  'quantity': 1000},
 {'bnf_code': '0101021B0AAAHAH',
  'items': 1,
  'practice': 'N81013',
  'bnf_name': 'Alginate_Raft-Forming Oral Susp S/F',
  'nic': 1.95,
  'act_cost': 1.82,
  'quantity': 500}]

A [glossary of terms](http://webarchive.nationalarchives.gov.uk/20180328130852tf_/http://content.digital.nhs.uk/media/10686/Download-glossary-of-terms-for-GP-prescribing---presentation-level/pdf/PLP_Presentation_Level_Glossary_April_2015.pdf/) and [FAQ](http://webarchive.nationalarchives.gov.uk/20180328130852tf_/http://content.digital.nhs.uk/media/10048/FAQs-Practice-Level-Prescribingpdf/pdf/PLP_FAQs_April_2015.pdf/) is available from the NHS regarding the data. Below we supply a data dictionary briefly describing what these fields mean.

| Data field |Description|
|:----------:|-----------|
|`'practice'`|Code designating the medical practice issuing the prescription|
|`'bnf_code'`|British National Formulary drug code|
|`'bnf_name'`|British National Formulary drug name|
|`'quantity'`|Number of capsules/quantity of liquid/grams of powder prescribed|
| `'items'`  |Number of refills (e.g. if `'quantity'` is 30 capsules, 3 `'items'` means 3 bottles of 30 capsules)|
|  `'nic'`   |Net ingredient cost|
|`'act_cost'`|Total cost including containers, fees, and discounts|

The `practices` variable is a list of member medical practices of the NHS. Each practice is represented by a dictionary containing identifying information for the medical practice. Most of the data fields are self-explanatory. Notice the values in the `'code'` field of `practices` match the values in the `'practice'` field of `scripts`.

In [8]:
practices[:2]

[{'code': 'A81001',
  'name': 'THE DENSHAM SURGERY',
  'addr_1': 'THE HEALTH CENTRE',
  'addr_2': 'LAWSON STREET',
  'borough': 'STOCKTON ON TEES',
  'village': 'CLEVELAND',
  'post_code': 'TS18 1HU'},
 {'code': 'A81002',
  'name': 'QUEENS PARK MEDICAL CENTRE',
  'addr_1': 'QUEENS PARK MEDICAL CTR',
  'addr_2': 'FARRER STREET',
  'borough': 'STOCKTON ON TEES',
  'village': 'CLEVELAND',
  'post_code': 'TS18 2AW'}]

In the following questions we will ask you to explore this data set. You may need to combine pieces of the data set together in order to answer some questions. Not every element of the data set will be used in answering the questions.

## Question 1: summary_statistics

Our beneficiary data (`scripts`) contains quantitative data on the number of items dispensed (`'items'`), the total quantity of item dispensed (`'quantity'`), the net cost of the ingredients (`'nic'`), and the actual cost to the patient (`'act_cost'`). Whenever working with a new data set, it can be useful to calculate summary statistics to develop a feeling for the volume and character of the data. This makes it easier to spot trends and significant features during further stages of analysis.

Calculate the sum, mean, standard deviation, and quartile statistics for each of these quantities. Format your results for each quantity as a list: `[sum, mean, standard deviation, 1st quartile, median, 3rd quartile]`. We'll create a `tuple` with these lists for each quantity as a final result.

In [9]:
len(scripts)

382726

In [10]:
from math import sqrt
def describe(key):
    
    data = [scripts[j][key] for j in range(len(scripts))]
            
    total = sum(data)
    avg = total/len(data)
    s = sqrt(sum([(j-avg)**2 for j in data])/len(data))
    w = sorted(data)
    q25 = w[len(w)//4]
    med = w[len(w)//2]
    q75 = w[(len(w)*3)//4]

    return (total, avg, s, q25, med, q75)

In [11]:
summary = [('items', describe('items')),
           ('quantity', describe('quantity')),
           ('nic', describe('nic')),
           ('act_cost', describe('act_cost'))]

In [12]:
grader.score.pw__summary_statistics(summary)

Your score:  1.0


## Question 2: most_common_item

Often we are not interested only in how the data is distributed in our entire data set, but within particular groups -- for example, how many items of each drug (i.e. `'bnf_name'`) were prescribed? Calculate the total items prescribed for each `'bnf_name'`. What is the most commonly prescribed `'bnf_name'` in our data?

To calculate this, we first need to split our data set into groups corresponding with the different values of `'bnf_name'`. Then we can sum the number of items dispensed within in each group. Finally we can find the largest sum.

We'll use `'bnf_name'` to construct our groups. You should have *5619* unique values for `'bnf_name'`.

In [13]:
bnf_names = {i['bnf_name'] for i \
             in scripts}
assert(len(bnf_names) == 5619)

We want to construct "groups" identified by `'bnf_name'`, where each group is a collection of prescriptions (i.e. dictionaries from `scripts`). We'll construct a dictionary called `groups`, using `bnf_names` as the keys. We'll represent a group with a `list`, since we can easily append new members to the group. To split our `scripts` into groups by `'bnf_name'`, we should iterate over `scripts`, appending prescription dictionaries to each group as we encounter them.

In [14]:
groups = {name: [] for name in \
          bnf_names}
for script in scripts:
    groups[script['bnf_name']\
          ].append(script)

In [15]:
groups[list(groups.keys())[0]][:2]

[{'bnf_code': '0109010U0AAAAAA',
  'items': 1,
  'practice': 'N81085',
  'bnf_name': 'Ursodeoxycholic Acid_Tab 150mg',
  'nic': 28.53,
  'act_cost': 26.52,
  'quantity': 90},
 {'bnf_code': '0109010U0AAAAAA',
  'items': 2,
  'practice': 'N81088',
  'bnf_name': 'Ursodeoxycholic Acid_Tab 150mg',
  'nic': 106.52,
  'act_cost': 98.84,
  'quantity': 336}]

Now that we've constructed our groups we should sum up `'items'` in each group and find the `'bnf_name'` with the largest sum. The result, `max_item`, should have the form `[(bnf_name, item total)]`, e.g. `[('Foobar', 2000)]`.

In [16]:
def sum_field(groups, field):
    items_sum = []
    for name in groups:
        item_sum = 0
        for script in groups[name]:
            item_sum += script[field]
        items_sum.append((name, \
                          item_sum))
    return items_sum

In [17]:
max_item = [sorted(sum_field(groups,\
'items'), key=lambda x: \
x[1], reverse=True)[0]] 


**TIP:** If you are getting an error from the grader below, please make sure your answer conforms to the correct format of `[(bnf_name, item total)]`.

In [18]:
grader.score.pw__most_common_item(max_item)

Your score:  1.0


**Challenge:** Write a function that constructs groups as we did above. The function should accept a list of dictionaries (e.g. `scripts` or `practices`) and a tuple of fields to `groupby` (e.g. `('bnf_name')` or `('bnf_name', 'post_code')`) and returns a dictionary of groups. The following questions will require you to aggregate data in groups, so this could be a useful function for the rest of the miniproject.

In [19]:
def group_by_field(data, fields):
    unique_field = {tuple([i[j] for \
j in fields]) for i in data}
    
    groups = {name: [] for name in \
          unique_field}
    
    for d in data:
        group = tuple([d[i] for i \
                in fields])
        groups[group].append(d)
        
    if len(fields) == 1:
        return {i[0]: groups[i] for \
               i in groups} 
    return groups 


In [20]:
groups = group_by_field(scripts, \
                ('bnf_name',))

test_max_item = [sorted(sum_field(\
groups, 'items'), key=lambda x: \
x[1], reverse=True)[0]] 

assert test_max_item == max_item

## Question 3: postal_totals

Our data set is broken up among different files. This is typical for tabular data to reduce redundancy. Each table typically contains data about a particular type of event, processes, or physical object. Data on prescriptions and medical practices are in separate files in our case. If we want to find the total items prescribed in each postal code, we will have to _join_ our prescription data (`scripts`) to our clinic data (`practices`).

Find the total items prescribed in each postal code, representing the results as a list of tuples `(post code, total items prescribed)`. Sort your results ascending alphabetically by post code and take only results from the first 100 post codes. Only include post codes if there is at least one prescription from a practice in that post code.

**NOTE:** Some practices have multiple postal codes associated with them. Use the alphabetically first postal code.

We can join `scripts` and `practices` based on the fact that `'practice'` in `scripts` matches `'code'` in `practices`. However, we must first deal with the repeated values of `'code'` in `practices`. We want the alphabetically first postal codes.

In [21]:
practice_postal = {}
for practice in practices:
    if practice['code'] in \
practice_postal:
        practice_postal[\
practice['code']] = min([\
practice_postal[practice['code']], \
practice['post_code']])
    else:
        practice_postal[\
practice['code']] = \
 practice['post_code']
        

In [22]:
list(practice_postal.items())[:2]
               

[('A81001', 'TS18 1HU'), ('A81002', 'TS18 2AW')]

**Challenge:** This is an aggregation of the practice data grouped by practice codes. Write an alternative implementation of the above cell using the `group_by_field` function you defined previously.

In [23]:
assert practice_postal['K82019'] == 'HP21 8TR'

Now we can join `practice_postal` to `scripts`.

In [24]:
joined = scripts[:]
for script in joined:
    script['post_code'] = \
practice_postal[script['practice']]
    

In [25]:
joined[:2]

[{'bnf_code': '0101010G0AAABAB',
  'items': 2,
  'practice': 'N81013',
  'bnf_name': 'Co-Magaldrox_Susp 195mg/220mg/5ml S/F',
  'nic': 5.98,
  'act_cost': 5.56,
  'quantity': 1000,
  'post_code': 'SK11 6JL'},
 {'bnf_code': '0101021B0AAAHAH',
  'items': 1,
  'practice': 'N81013',
  'bnf_name': 'Alginate_Raft-Forming Oral Susp S/F',
  'nic': 1.95,
  'act_cost': 1.82,
  'quantity': 500,
  'post_code': 'SK11 6JL'}]

Finally we'll group the prescription dictionaries in `joined` by `'post_code'` and sum up the items prescribed in each group, as we did in the previous question.

In [47]:
dict([('a', 1), ('b', 2)])

{'a': 1, 'b': 2}

In [53]:
items_by_post = group_by_field(\
    joined, ('post_code',))

dict([list(items_by_post.items())[0]\
     ])


{'DN22 7XF': [{'bnf_code': '0101021B0AAAHAH',
   'items': 5,
   'practice': 'C84013',
   'bnf_name': 'Alginate_Raft-Forming Oral Susp S/F',
   'nic': 25.67,
   'act_cost': 23.82,
   'quantity': 2500,
   'post_code': 'DN22 7XF'},
  {'bnf_code': '0101021B0AAALAL',
   'items': 119,
   'practice': 'C84013',
   'bnf_name': 'Sod Algin/Pot Bicarb_Susp S/F',
   'nic': 685.46,
   'act_cost': 636.62,
   'quantity': 66940,
   'post_code': 'DN22 7XF'},
  {'bnf_code': '0101021B0AAAPAP',
   'items': 3,
   'practice': 'C84013',
   'bnf_name': 'Sod Alginate/Pot Bicarb_Tab Chble 500mg',
   'nic': 9.21,
   'act_cost': 8.56,
   'quantity': 180,
   'post_code': 'DN22 7XF'},
  {'bnf_code': '0101021B0BEADAJ',
   'items': 4,
   'practice': 'C84013',
   'bnf_name': 'Gaviscon Infant_Sach 2g (Dual Pack) S/F',
   'nic': 33.09,
   'act_cost': 30.88,
   'quantity': 103,
   'post_code': 'DN22 7XF'},
  {'bnf_code': '0101021B0BEAIAL',
   'items': 61,
   'practice': 'C84013',
   'bnf_name': 'Gaviscon Advance_Liq (Anis

In [78]:
postal_totals = sorted(sum_field(\
items_by_post, 'items'), \
key=lambda x: x[0])

grader.score.pw__postal_totals(postal_totals[:100])

Your score:  1.0


## Question 4: items_by_region

Now we'll combine the techniques we've developed to answer a more complex question. Find the most commonly dispensed item in each postal code, representing the results as a list of tuples (`post_code`, `bnf_name`, amount dispensed as proportion of total). Sort your results ascending alphabetically by post code and take only results from the first 100 post codes.

**NOTE:** We'll continue to use the `joined` variable we created before, where we've chosen the alphabetically first postal code for each practice. Additionally, some postal codes will have multiple `'bnf_name'` with the same number of items prescribed for the maximum. In this case, we'll take the alphabetically first `'bnf_name'`.

Now we need to calculate the total items of each `'bnf_name'` prescribed in each `'post_code'`. Use the techniques we developed in the previous questions to calculate these totals. You should have 141196 `('post_code', 'bnf_name')` groups.

In [69]:
total_items_by_bnf_post = sum_field(\
group_by_field(joined, \
('post_code', 'bnf_name')), 'items')
assert len(total_items_by_bnf_post) == 141196

In [70]:
total_items_by_bnf_post[:3]

[(('DN34 4GB', 'Jobst Elvarex Soft For L/Extrem Class 1'), 10),
 (('SK6 1ND', 'Liraglutide_Inj 6mg/ml 3ml PF Pen'), 5),
 (('RM3 9SU', 'E45_Emollient Bath Oil'), 2)]

Let's use `total_items` to find the maximum item total for each postal code. To do this, we will want to regroup `total_items_by_bnf_post` by `'post_code'` only, not by `('post_code', 'bnf_name')`. First let's turn `total_items` into a list of dictionaries (similar to `scripts` or `practices`) and then group it by `'post_code'`. You should have 118 groups in the resulting `total_items_by_post` after grouping `total_items` by `'post_code'`.

In [71]:
total_items = [{'post_code': i[0][0], \
'bnf_name': i[0][1], 'items': i[1]} \
for i in total_items_by_bnf_post]

total_items[:3]

[{'post_code': 'DN34 4GB',
  'bnf_name': 'Jobst Elvarex Soft For L/Extrem Class 1',
  'items': 10},
 {'post_code': 'SK6 1ND',
  'bnf_name': 'Liraglutide_Inj 6mg/ml 3ml PF Pen',
  'items': 5},
 {'post_code': 'RM3 9SU', 'bnf_name': 'E45_Emollient Bath Oil', 'items': 2}]

In [72]:
total_items_by_post = \
group_by_field(total_items, \
('post_code',))

dict([list(\
total_items_by_post.items())[0]\
     ])

{'DN22 7XF': [{'post_code': 'DN22 7XF',
   'bnf_name': 'Dapagliflozin_Tab 10mg',
   'items': 33},
  {'post_code': 'DN22 7XF', 'bnf_name': 'Indometacin_Cap 50mg', 'items': 7},
  {'post_code': 'DN22 7XF',
   'bnf_name': 'Allevyn Gentle Border 10cm x 20cm Wound',
   'items': 3},
  {'post_code': 'DN22 7XF', 'bnf_name': 'Primidone_Tab 250mg', 'items': 5},
  {'post_code': 'DN22 7XF', 'bnf_name': 'Allopurinol_Tab 100mg', 'items': 150},
  {'post_code': 'DN22 7XF',
   'bnf_name': 'Ins Humalog_100u/ml 3ml Cart',
   'items': 17},
  {'post_code': 'DN22 7XF', 'bnf_name': 'Alzain_Cap 300mg', 'items': 2},
  {'post_code': 'DN22 7XF',
   'bnf_name': 'LoFric Primo Nelaton Male NonPVC 8-18(30',
   'items': 3},
  {'post_code': 'DN22 7XF',
   'bnf_name': 'Co-Careldopa_Tab 25mg/100mg',
   'items': 13},
  {'post_code': 'DN22 7XF', 'bnf_name': 'Kapake_Cap 30mg/500mg', 'items': 3},
  {'post_code': 'DN22 7XF', 'bnf_name': 'Clobet But_Oint 0.05%', 'items': 12},
  {'post_code': 'DN22 7XF',
   'bnf_name': 'Clinita

In [73]:
assert len(total_items_by_post) == 118

Now we will aggregate the groups in `total_by_item_post` to create `max_item_by_post`. Some `'bnf_name'` have the same item total within a given postal code. Therefore, if more than one `'bnf_name'` has the maximum item total in a given postal code, we'll take the alphabetically first `'bnf_name'`. We can do this by [sorting](https://docs.python.org/2.7/howto/sorting.html) each group according to the item total and `'bnf_name'`.

In [76]:
max_item_by_post = {k: sorted(\
total_items_by_post[k], \
key=lambda x: (-x['items'], \
x['bnf_name']))[0] for k in \
total_items_by_post}                   

In [77]:
dict(list(max_item_by_post.items()\
)[:3])


{'DN22 7XF': {'post_code': 'DN22 7XF',
  'bnf_name': 'Simvastatin_Tab 40mg',
  'items': 857},
 'OL1 1NL': {'post_code': 'OL1 1NL',
  'bnf_name': 'Omeprazole_Cap E/C 20mg',
  'items': 1372},
 'LN2 2JP': {'post_code': 'LN2 2JP',
  'bnf_name': 'Omeprazole_Cap E/C 20mg',
  'items': 1671}}

In order to express the item totals as a proportion of the total amount of items prescribed across all `'bnf_name'` in a postal code, we'll need to use the total items prescribed that we previously calculated as `items_by_post`. Calculate the proportions for the most common `'bnf_names'` for each postal code. Format your answer as a list of tuples: `[(post_code, bnf_name, total)]`

In [82]:
items_by_region = sorted([(p, \
max_item_by_post[p]['bnf_name'], \
max_item_by_post[p]['items'] / \
dict(postal_totals)[p]) for p in \
max_item_by_post], lambda x: x[0]\
                )[:100]

TypeError: sorted expected 1 arguments, got 2

In [ ]:
grader.score.pw__items_by_region(items_by_region)

*Copyright &copy; 2020 The Data Incubator.  All rights reserved.*